In [1]:
import numpy
# import scipy.io as scio
import hdf5storage as hdf5
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os,re
from tqdm import tqdm

import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from module import rgnn

In [4]:
def EmotionDL(label,noise = 0.6):
    if label == 0:
        new_label = [1 - 2 * noise / 3, 2 * noise / 3, 0]
    elif label == 1:
        new_label = [noise / 3, 1 - 2 * noise / 3, noise / 3]
    elif label == 2:
        new_label = [0, 2 * noise / 3, 1 - 2 * noise / 3]
    return new_label
def extend_normal(sample):
    for i in range(len(sample)):
        features_min = np.min(sample[i])
        features_max = np.max(sample[i])
        sample[i] = (sample[i] - features_min) / (features_max - features_min)
    return sample

seed_path = 'seed_processed'
data_path = os.path.join(seed_path, 'dic_all_subjects.mat')
label_path = os.path.join(seed_path, 'dic_all_subjects_label.mat')
data_dic = hdf5.loadmat(data_path)
label_dic = hdf5.loadmat(label_path)
for key in data_dic.keys():
    if key.startswith('__'):
        continue
    data_dic[key] = extend_normal(data_dic[key]) # 归一化; 2025*62*5
    label_dic[key] = np.array([EmotionDL(label) for label in label_dic[key][0]]) # 1*2025 -> 2025*3
    print(key, data_dic[key].shape, label_dic[key].shape)

print(label_dic[key])

sub_1 (2025, 62, 25) (2025, 3)
sub_2 (2025, 62, 25) (2025, 3)
sub_3 (2025, 62, 25) (2025, 3)
sub_4 (2025, 62, 25) (2025, 3)
sub_5 (2025, 62, 25) (2025, 3)
sub_6 (2025, 62, 25) (2025, 3)
sub_7 (2025, 62, 25) (2025, 3)
sub_8 (2025, 62, 25) (2025, 3)
sub_9 (2025, 62, 25) (2025, 3)
sub_10 (2025, 62, 25) (2025, 3)
sub_11 (2025, 62, 25) (2025, 3)
sub_12 (2025, 62, 25) (2025, 3)
sub_13 (2025, 62, 25) (2025, 3)
sub_14 (2025, 62, 25) (2025, 3)
sub_15 (2025, 62, 25) (2025, 3)
[[0.  0.4 0.6]
 [0.  0.4 0.6]
 [0.  0.4 0.6]
 ...
 [0.6 0.4 0. ]
 [0.6 0.4 0. ]
 [0.6 0.4 0. ]]


In [10]:
def sub_split(data_dict, label_dict, sub_test, sub_train=None):
    
    if sub_test not in range(1, 16):
        raise ValueError('sub for test must be in range(1, 16)')
    if sub_train is not None:
        if all(1 <= x <= 16 for x in sub_train)==False:
            raise ValueError('sub for train must be in range(1, 16)')
        if sub_test in sub_train:
            raise ValueError('sub for test and train can not overlap')
    
    x_train =[]
    y_train = []
    x_test =None
    y_test = None
    train_data_list = []
    test_data_list = [] 
    train_sub_list = [] # 训练集的被试列表
    for i in range(1, 16):
        for key in data_dict.keys():
            if key.startswith('__'): continue
            sub_num = int(key.split('_')[1])
            if sub_num != i: continue
            # print('shape:', key, data_dict[key].shape, label_dict[key].shape)
            if sub_num != sub_test:
                if sub_train is not None:
                    if sub_num in sub_train:
                        x_train.append(data_dict[key])
                        y_train.append(label_dict[key])
                        train_sub_list.append(key)
                    else: pass
                else:
                    x_train.append(data_dict[key])
                    y_train.append(label_dict[key])
                    train_sub_list.append(key)
            else:
                print('{} for test sub'.format(key))
                x_test = data_dict[key]
                y_test = label_dict[key]
    print('train sub list:', train_sub_list)
    
    x_train = np.asarray(x_train)
    y_train = np.asarray(y_train) 
    x_test = np.asarray(x_test)
    y_test = np.asarray(y_test)
    
    x_train = x_train.reshape(-1, x_train.shape[-2], x_train.shape[-1])
    y_train = y_train.reshape(-1,y_train.shape[-1])
    x_test = x_test.reshape(-1, x_test.shape[-2], x_test.shape[-1])
    y_test = y_test.reshape(-1,y_test.shape[-1])
    print(f'x_train:{x_train.shape},  y_train:{y_train.shape},  x_test:{x_test.shape},  y_test:{y_test.shape}')
    
    for i in range(len(x_train)):
        train_data_list.append(Data(x=torch.from_numpy(x_train[i]).float(), edge_index=edge_index, edge_attr=edge_weight, y=torch.Tensor(y_train[i])))
    for i in range(len(x_test)):
        test_data_list.append(Data(x=torch.from_numpy(x_test[i]).float(), edge_index=edge_index, edge_attr=edge_weight, y=torch.Tensor(y_test[i])))
    
    assert(len(train_data_list)==x_train.shape[0] and len(train_data_list)==y_train.shape[0])
    assert(len(test_data_list)==x_test.shape[0] and len(test_data_list)==y_test.shape[0])
    
    return train_data_list, test_data_list
    
sub = 13 # 1-15
    
sub_list = [i for i in range(1, 16) if i != sub]
train_data_list, test_data_list = sub_split(data_dic, label_dic, sub, sub_list)
print(len(train_data_list), len(test_data_list))

batch_size = 128
train_loader = DataLoader(train_data_list, batch_size=batch_size)
test_loader = DataLoader(test_data_list, batch_size=batch_size)
for batch_data in train_loader:
    print('batch data in trainer:')
    print(batch_data)
    break
for batch_data in test_loader:
    print('batch data in tester:')
    print(batch_data)
    break

sub_13 for test sub
train sub list: ['sub_1', 'sub_2', 'sub_3', 'sub_4', 'sub_5', 'sub_6', 'sub_7', 'sub_8', 'sub_9', 'sub_10', 'sub_11', 'sub_12', 'sub_14', 'sub_15']
x_train:(28350, 62, 25),  y_train:(28350, 3),  x_test:(2025, 62, 25),  y_test:(2025, 3)


NameError: name 'edge_index' is not defined